### Multiscale Spatiotemporal Vision Transformer (MST-ViT | PixFormer)

### Misael M. Morales, 2024
***

In [1]:
from main import *

hete = Heterogeneity()


------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.1.0+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------



In [ ]:
hete.make_dataloaders()

In [62]:
tsteps = 60
orig_img = 256

for i in range(1000):
    data = np.load('Fdataset/sample_{}.npz'.format(i+1))
    poro  = np.tile(data['poro'], (60, 1, 1, 1))
    perm  = np.tile(data['perm'], (60, 1, 1, 1))
    tstep = np.tile(np.arange(1, tsteps+1).reshape(tsteps, 1, 1, 1), (1, 1, orig_img, orig_img))
    X_data = np.concatenate([poro, perm, tstep], axis=1)
    
    pres = np.expand_dims(data['pres'], axis=1)
    satu = np.expand_dims(data['sat'], axis=1)
    y_data = np.concatenate([pres, satu], axis=1)

    np.savez('train_dataF/realization_{}.npz'.format(i+1), X_data=X_data, y_data=y_data)

In [ ]:
hete.trainer()

In [ ]:
hete.tester()

In [ ]:
hete.plot_losses()

***
# END